## TODO List

#### Open Questions
- [ ] t-SNE with test or train samples?

#### Training
- [x] MLP 
- [ ] Different Models / Datasets

#### Embedding Visualization
Data
- [x] PCA
- [ ] Use ALL train data for PCA?
- [x] t-SNE
- [ ] Visualize t-SNE training Steps
- [x] UMAP
- [x] UMAP Parameters
Visualization
- [ ] Live during training
- [ ] 3D
- [ ] With trajectory

#### Loss Landscape Visualization (later)
- [x] Get Code
- [ ] Classical Loss Landscape
- [ ] Topological Loss Landscape

# Visualizing Embedding Evolution During Training

In this notebook, we train an MLP on MNIST and visualize how hidden representations (embeddings) evolve over training using t-SNE. This gives insight into how class structure emerges in the latent space over time.

In [ ]:
%matplotlib widget

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random

# Reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(torch.version.cuda)  # Should print something like '11.8'
print("Using device:", device)

# Dataset
### MNIST

In [ ]:
transform = transforms.ToTensor()

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

### Test Subset for visualization

In [ ]:
from torch.utils.data import Subset

# First, get all test labels
test_targets = test_data.targets.numpy()

# Define how many samples per class
samples_per_class = 100
num_classes = 10

# Collect indices (balanced subset)
selected_indices = []
for class_id in range(num_classes):
    class_indices = np.where(test_targets == class_id)[0]
    chosen = np.random.choice(class_indices, size=samples_per_class, replace=False)
    selected_indices.extend(chosen)

# Create a test subset DataLoader (shuffling not needed)
test_subset = Subset(test_data, selected_indices)
test_subset_loader = DataLoader(test_subset, batch_size=128, shuffle=False)

In [ ]:
print(len(train_data))
print(len(test_data))
print(len(test_subset))

# Model
### MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim=28*28, hidden_dim=128, num_classes=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, return_embedding=False):
        x = x.view(x.size(0), -1)
        h = F.relu(self.fc1(x))
        out = self.fc2(h)
        if return_embedding:
            return out, h
        return out

model_mlp = MLP().to(device)

In [ ]:
print(f"{len(train_loader)} Batches")

##### Training
After 10 Epochs
| Embedding Dimension | Final Train Acc | Final Test Acc | Test Epoch 3 |
| ------------- | ------------- | ------------- | ------------- |
| 128 | 0.9897 | 0.9760 | 0.9610 |
| 64 | 0.9782 | **0.9706** | 0.9504 |
| 32 | 0.9678 | 0.9625 | 0.9428 |
| 16 | 0.9434 | 0.9398 | 0.9256 |

In [ ]:
%matplotlib inline

model_mlp = MLP(hidden_dim=32).to(device)

from train_viz import train_model_with_embedding_tracking

results = train_model_with_embedding_tracking(
    model=model_mlp,
    train_loader=train_loader,
    test_loader=test_loader,
    subset_loader=test_subset,
    device=device,
    epochs=10,
    learning_rate=0.0001, # 0.0001
    embedding_records_per_epoch=10,
    track_embedding_drift=True
)

##    return {
##        'train_losses': train_losses,
##        'val_losses': val_losses,
##        'train_accuracies': train_accuracies,
##        'val_accuracies': val_accuracies,
##        'test_subset_embeddings': test_subset_embeddings,
##        'test_subset_labels': test_subset_labels
##    }

### Embedding Drift
The embedding drift shows the Euclidean distance between the current embedding, compared to the snapshot of N (1 to 5) periods before.

It shows that the changes drop quickly, and that drifts measured over higher distances are in a similar range to distance 1.

In [ ]:
%matplotlib widget
from train_viz import _plot_embedding_drift

embedding_drifts = results["embedding_drifts"]

fig, axs = plt.subplots(1, 1, figsize=(10, 4))

# Plot 2x Drifts
axs.plot(range(1, len(embedding_drifts[1]) + 1), np.array(embedding_drifts[1]) * 2, color="green", label="2x Drift 1", alpha=0.3)
axs.plot(range(1, len(embedding_drifts[2]) + 1), np.array(embedding_drifts[2]) * 2, color="blue", label="2x Drift 2", alpha=0.3)

# Plot original drifts
_plot_embedding_drift(axs, embedding_drifts)

plt.legend()
plt.show()

# Video Animation
New

In [ ]:
from train_viz import generate_projections, animate_projections, show_with_slider, show_multiple_projections_with_slider

## PCA

In [ ]:
projections_pca_first = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='pca',
    pca_fit_basis='first',
)
projections_pca_last = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='pca',
    pca_fit_basis='last',
)
projections_pca_all = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='pca',
    pca_fit_basis='all',
)

In [ ]:
%matplotlib widget

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_pca_first, projections_pca_last, projections_pca_all],
    labels=results["subset_labels"],
    titles=["PCA on first", "PCA on last", "PCA on all"],
    interpolate=True,
    steps_per_transition=2,
    figsize_per_plot=(4, 4),
)

In [ ]:
from IPython.display import HTML
ani_pca = animate_projections(
    projections_pca_all,
    labels=results["subset_labels"],
    interpolate=False,
    steps_per_transition=0
)
HTML(ani_pca.to_jshtml())

## tSNE

In [ ]:
projections_tsne = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='tsne',
)

In [ ]:
ani_tsne = show_with_slider(
    projections_tsne,
    labels=results["subset_labels"],
    interpolate=False,
)

## t-SNE Backwards (reverse computation)

In [ ]:
projections_tsne_reverse = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='tsne',
    reverse_computation=True
)

In [ ]:
ani_tsne_reverse = show_with_slider(
    projections_tsne_reverse,
    labels=results["subset_labels"],
    interpolate=False,
)

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_tsne, projections_tsne_reverse],
    labels=results["subset_labels"],
    titles=["t-SNE", "t-SNE backwards"],
    interpolate=True,
    steps_per_transition=5,
    figsize_per_plot=(5, 5),
)

## t-SNE distance metric
euclidean vs cosine

In [ ]:
projections_tsne_cosine = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='tsne',
    metric='cosine'
)

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_tsne, projections_tsne_cosine],
    labels=results["subset_labels"],
    titles=["t-SNE", "t-SNE Cosine"],
    interpolate=True,
    steps_per_transition=5,
    figsize_per_plot=(5, 5),
)

## t-SNE Seeds

In [ ]:
projections_tsne_2 = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='tsne',
    random_state=1106,
    tsne_init='random'
)

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_tsne, projections_tsne_2, projections_tsne_reverse],
    labels=results["subset_labels"],
    titles=["t-SNE init with pca", "t-SNE init random", "t-SNE backwards"],
    steps_per_transition=5,
    figsize_per_plot=(4, 4),
)

## t-SNE Perplexity
Compare 5 - 10 - 30 - 50

In [ ]:
projections_tsne_p_5 = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='tsne',
    tsne_perplexity=5
)

projections_tsne_p_10 = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='tsne',
    tsne_perplexity=10
)

projections_tsne_p_50 = generate_projections(
    embeddings_list=results["subset_embeddings"],
    method='tsne',
    tsne_perplexity=50
)

In [ ]:
projections_tsne_p_30 = projections_tsne

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_tsne_p_5, projections_tsne_p_10, projections_tsne_p_30, projections_tsne_p_50],
    labels=results["subset_labels"],
    titles=["t-SNE Perplexity 5", "t-SNE Perplexity 10", "t-SNE Perpl. 30 (standard)", "t-SNE Perplexity 50"],
    interpolate=True,
    steps_per_transition=3,
    figsize_per_plot=(3.5, 3.5),
    dot_size=2,
    shared_axes=False
)

## t-SNE vs PCA

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_pca_all, projections_tsne],
    labels=results["subset_labels"],
    titles=["PCA on all", "t-SNE"],
    interpolate=True,
    steps_per_transition=10,
    figsize_per_plot=(5, 5),
    shared_axes=False,
)

In [ ]:
# ani_tsne = animate_projections(
#     projections_tsne,
#     labels=results["subset_labels"],
#     interpolate=True,
#     steps_per_transition=2
# )
# HTML(ani_tsne.to_jshtml())

## UMAP
UMAP (Uniform Manifold Approximation and Projection) is a **faster** alternative to t-SNE that also **aims to preserve some global structure**.

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    projections_umap = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='last',
        umap_n_neighbors=15,
    )

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_umap, projections_tsne],
    labels=results["subset_labels"],
    titles=["UMAP", "t-SNE"],
    interpolate=False,
    steps_per_transition=10,
    figsize_per_plot=(5, 5),
    shared_axes=False,
)

## UMAP Initialization
UMAP builds a graph-based representation of the data during `fit()`, learning the local and global structure (neighborhoods) in the `basis_data`. Once this structure is learned, it can be used to **transform new data** into the same low-dimensional space without retraining — unlike t-SNE, which must be refit every time.

This makes UMAP more efficient and suitable for dynamic or streaming embeddings.

IDEA: Fit not only on selection of test data?

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    projections_umap_first = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='first',
        umap_n_neighbors=15,
    )

In [ ]:
# Todo: Middle
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    projections_umap_first = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis=int(len(results["subset_embeddings"]) / 2),
        umap_n_neighbors=15,
    )

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_umap, projections_umap_first],
    labels=results["subset_labels"],
    titles=["UMAP", "UMAP on first"],
    interpolate=False,
    steps_per_transition=10,
    figsize_per_plot=(5, 5),
    shared_axes=False,
)

## UMAP Hyperparameters
#### Number of Neighbors
Controls balance between local vs. global structure. (low = local, high = global)

Usually `5`, `10`, **`15` (standard)**, `30`, `50`, `100`

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    projections_umap_n_5 = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='last',
        umap_n_neighbors=5,
    )

    projections_umap_n_50 = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='last',
        umap_n_neighbors=50,
    )

    projections_umap_n_100 = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='last',
        umap_n_neighbors=100,
    )

In [ ]:
projections_umap_n_15 = projections_umap

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_umap_n_5, projections_umap_n_15, projections_umap_n_50, projections_umap_n_100],
    labels=results["subset_labels"],
    titles=["UMAP 5 Neighbors", "UMAP 15 Neighbors", "UMAP 50 Neighbors", "UMAP 100 Neighbors"],
    figsize_per_plot=(4, 4),
    dot_size=3,
    shared_axes=False,
)

#### Min Distance
Controls how tightly UMAP packs points together in low-D space.
- Lower = tighter clusters, less overlap.
- Higher = looser structure, more space between points.

Try values: `0.01`, `0.05`, **`0.1`** (standard), `0.3`, `0.5`, 

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    projections_umap_d_001 = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='last',
        umap_n_neighbors=15,
        umap_min_dist=0.01
    )

    projections_umap_d_01 = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='last',
        umap_n_neighbors=15,
        umap_min_dist=0.1
    )

    projections_umap_d_05 = generate_projections(
        embeddings_list=results["subset_embeddings"],
        method='umap',
        pca_fit_basis='last',
        umap_n_neighbors=15,
        umap_min_dist=0.5
    )

In [ ]:
show_multiple_projections_with_slider(
    projections_list=[projections_umap_d_001, projections_umap_d_01, projections_umap_d_05],
    labels=results["subset_labels"],
    titles=["UMAP min_dist 0.01", "UMAP min_dist 0.1", "UMAP min_dist 0.5"],
    steps_per_transition=3,
    figsize_per_plot=(4, 4),
    dot_size=2,
)

# Embedding Trajectory Visualization

In [ ]:
%matplotlib widget
show_with_slider_and_trajectories(
    projections=projections_pca_all,
    labels=results["subset_labels"],
    interpolate=True,
    steps_per_transition=3,
    figsize=(5, 5),
    dot_size=5,
    trail_length=10,
    show_trajectories=True,
    point_stride=1,
    trail_filter='percentile',          # 'all', 'top_k', 'percentile'
    trail_filter_value=10        # top-k count or top-X% percentile
)

In [ ]:
%matplotlib widget
show_with_slider_and_trajectories(
    projections=projections_umap,
    labels=results["subset_labels"],
    interpolate=True,
    steps_per_transition=3,
    figsize=(5, 5),
    dot_size=5,
    trail_length=4,
    show_trajectories=True,
    point_stride=1,
    trail_filter='percentile',          # 'all', 'top_k', 'percentile'
    trail_filter_value=10        # top-k count or top-X% percentile
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

def show_with_slider_and_trajectories(
    projections,
    labels,
    figsize=(5, 5),
    cmap='tab10',
    dot_size=5,
    alpha=0.5,
    interpolate=False,
    steps_per_transition=10,
    show_trajectories=True,
    trail_length=15,
    point_stride=1,
    trail_filter='all',          # 'all', 'top_k', 'percentile'
    trail_filter_value=50        # top-k count or top-X% percentile
):
    # Interpolation
    if interpolate:
        projections_interp = []
        for a, b in zip(projections[:-1], projections[1:]):
            for alpha_step in np.linspace(0, 1, steps_per_transition, endpoint=False):
                interp = (1 - alpha_step) * a + alpha_step * b
                projections_interp.append(interp)
        projections_interp.append(projections[-1])
    else:
        projections_interp = projections

    projections = np.array(projections_interp)
    n_frames, n_points, _ = projections.shape

    # Select points by stride
    indices = np.arange(0, n_points, point_stride)
    n_active = len(indices)

    # --- Compute total movement per point ---
    displacements = np.linalg.norm(np.diff(projections, axis=0), axis=2)
    total_movement = displacements.sum(axis=0)

    # Determine which points to show tails for
    if trail_filter == 'all':
        tail_indices = set(indices)
    elif trail_filter == 'top_k':
        sorted_indices = np.argsort(total_movement)[::-1]
        tail_indices = set(sorted_indices[:trail_filter_value])
    elif trail_filter == 'percentile':
        cutoff = np.percentile(total_movement, 100 - trail_filter_value)
        tail_indices = set(np.where(total_movement >= cutoff)[0])
    else:
        raise ValueError(f"Invalid trail_filter: {trail_filter}")

    # Setup figure
    fig, ax = plt.subplots(figsize=figsize)
    all_proj = np.concatenate(projections, axis=0)
    max_abs = np.max(np.abs(all_proj))
    ax.set_xlim(-max_abs, max_abs)
    ax.set_ylim(-max_abs, max_abs)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])

    # Scatter
    scatter = ax.scatter([], [], c=[], s=dot_size, cmap=cmap, alpha=alpha, zorder=2)

    # Trails
    trail_lines = []
    if show_trajectories:
        for i in indices:
            segments = []
            if i in tail_indices:
                for _ in range(trail_length):
                    line, = ax.plot([], [], lw=1.0, alpha=0.0, color='gray', zorder=1)
                    segments.append(line)
            trail_lines.append(segments)

    # --- Update function ---
    def update(frame_idx):
        current_coords = projections[frame_idx][indices]
        current_labels = np.array(labels[0])[indices]
        scatter.set_offsets(current_coords)
        scatter.set_array(current_labels)

        if show_trajectories and frame_idx > 0:
            for i, point_idx in enumerate(indices):
                segments = trail_lines[i]
                if not segments:  # point wasn't selected for trail
                    continue
                for j in range(trail_length):
                    t = frame_idx - j - 1
                    if t < 0:
                        segments[j].set_alpha(0.0)
                        continue
                    p1 = projections[t, point_idx]
                    p2 = projections[t + 1, point_idx]
                    segments[j].set_data([p1[0], p2[0]], [p1[1], p2[1]])
                    
                    # Fade oldest segment the most
                    alpha = (trail_length - j) / trail_length * 0.5
                    segments[j].set_alpha(alpha)
                    
        fig.canvas.draw_idle()

    # --- Slider widgets ---
    slider = widgets.Play(min=0, max=n_frames - 1, step=1)
    slider_control = widgets.IntSlider(min=0, max=n_frames - 1, step=1)
    widgets.jslink((slider, 'value'), (slider_control, 'value'))

    out = widgets.interactive_output(update, {'frame_idx': slider_control})
    display(widgets.VBox([widgets.HBox([slider, slider_control]), out]))


# Video Export

In [ ]:
# === Save as MP4 (with ffmpeg) ===
model_name = model._get_name()
lr = learning_rate
base_name = f"tsne_{model_name}_e{epochs}_lr{lr:.0e}"
i = 1
while os.path.exists(f"{base_name}_{i:03}.mp4"):
    i += 1
filename = f"{base_name}_{i:03}.mp4"

ani.save(filename, writer='ffmpeg', dpi=150)
print(f"Saved animation as {filename}")

In [ ]:
import os

# === Define experiment info ===
model_name = model._get_name()
# Build a base name with settings
base_name = f"tsne_{model_name}_e{epochs}_lr{learning_rate:.0e}"  # e.g., tsne_MLP_e10_lr1e-03

# Auto-increment file index
i = 1
while os.path.exists(f"{base_name}_{i:03}.gif"):
    i += 1
filename = f"{base_name}_{i:03}.gif"

# Save the animation
ani.save(filename, writer='pillow')
print(f"Saved animation as {filename}")

# Video Animation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.manifold import TSNE
import os

# === Interpolation between t-SNE results ===
def interpolate_tsne(embeddings_list, steps=10):
    interpolated = []
    for a, b in zip(embeddings_list[:-1], embeddings_list[1:]):
        for alpha in np.linspace(0, 1, steps, endpoint=False):
            interp = (1 - alpha) * a + alpha * b
            interpolated.append(interp)
    interpolated.append(embeddings_list[-1])  # include final frame
    return interpolated

# === Compute t-SNE per epoch using selected_indices ===
tsne_results = []

# Epoch 0
tsne = TSNE(n_components=2, init='pca', random_state=42)
result = tsne.fit_transform(embeddings[0][selected_indices])
tsne_results.append(result)

# Subsequent epochs
for epoch in range(1, len(embeddings)):
    tsne = TSNE(n_components=2, init=tsne_results[-1], random_state=42)
    result = tsne.fit_transform(embeddings[epoch][selected_indices])
    tsne_results.append(result)

# === Interpolate between epochs ===
tsne_results_interp = interpolate_tsne(tsne_results, steps=5)

# === Animation setup ===
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter([], [], cmap='tab10', alpha=0.7)
ax.set_xlim(-100, 100)
ax.set_ylim(-100, 100)
ax.set_xlabel('t-SNE Dimension 1')
ax.set_ylabel('t-SNE Dimension 2')
title = ax.set_title('t-SNE Embedding Evolution')

def update(frame):
    scatter.set_offsets(tsne_results_interp[frame])
    scatter.set_array(sampled_labels)
    title.set_text(f't-SNE Embedding Evolution – Frame {frame + 1}')
    return scatter,

ani = animation.FuncAnimation(fig, update, frames=len(tsne_results_interp), interval=100, blit=True)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
# === Save as MP4 (with ffmpeg) ===
model_name = model._get_name()
lr = learning_rate
base_name = f"tsne_{model_name}_e{epochs}_lr{lr:.0e}"
i = 1
while os.path.exists(f"{base_name}_{i:03}.mp4"):
    i += 1
filename = f"{base_name}_{i:03}.mp4"

ani.save(filename, writer='ffmpeg', dpi=150)
print(f"Saved animation as {filename}")